In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
face_cascade=cv2.CascadeClassifier(r'F:\face-mask-assignment\cascades\haarcascade_frontalface_default.xml')

In [3]:
model = load_model(r'F:\face-mask-assignment\model\face_mask_detection_model.h5')

In [ ]:
IMG_SIZE = 227
video=cv2.VideoCapture(0)
address = 'http://192.168.196.199:8080/video'
video.open(address)

while True:
    try:
        check,frame=video.read()
        grayFrame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(grayFrame)

        for x,y,w,h in faces:
            roi_gray = grayFrame[y:y+h,x:x+w]
            roi_color = frame[y:y+h,x:x+w]

            face = cv2.resize(roi_color,(IMG_SIZE,IMG_SIZE))
            exp = np.expand_dims(face,axis=0)
            prediction = model.predict(exp)
            pred = np.argmax(prediction)
            confidence ="{:.2f}".format(np.max(prediction)*100)

            if pred==0:
                res = 'Wearing a Mask'
                color=(0,255,0)
            else:
                res = 'Not wearing a Mask'
                color=(0,0,255)

            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(frame,res+':'+confidence+'%',(x,y-10),2,1,color,2)
        cv2.imshow('FaceDetection',frame)

        key=cv2.waitKey(1)
        if key==ord('x'):
            break
            
    except: 
        break               
        
video.release()
cv2.destroyWindow('FaceDetection')